In [1]:
import os

In [2]:
%pwd

'/home/saitama/deepl_chicken_disease_classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/saitama/deepl_chicken_disease_classification'

In [39]:
# configuration entity for data ingestion class

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_log_dir : Path
    checkpoint_model_filepath : Path

In [40]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [59]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        
        model_checkpoints_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(config.tensorboard_root_log_dir),
            Path(model_checkpoints_dir)
            ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callbacks_config

In [60]:
import tensorflow as tf
import time

In [63]:
class PrepareCallbacks:
    def __init__(self, config= PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tensorboard_callbacks(self):
        timestamps = time.strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,
                                          f"tb_log_at_{timestamps}",
                                          )
        return tf.keras.callbacks.TensorBoard(log_dir=tensorboard_running_log_dir)
    


    @property
    def _create_checkpoint_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath= str(self.config.checkpoint_model_filepath),
                                                  save_best_only= True
                                                  )
    

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tensorboard_callbacks,
            self._create_checkpoint_callbacks
        ]


In [64]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    list_callbacks = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-10-11 08:49:31,855: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-11 08:49:31,860: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-11 08:49:31,863: INFO: common: created directory at: artifacts]
[2023-10-11 08:49:31,865: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2023-10-11 08:49:31,867: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
